## Import librairies

In [2]:
import numpy as np
import cv2
import os
from PIL import Image
import matplotlib.pyplot as plt
import keras_ocr
import math

## Rognage

In [ ]:
image = cv2.imread("./constellations/leo/leo0.png")

In [ ]:
image.shape

In [ ]:
try:
    os.mkdir("preprocessed_1")
except:
    pass

In [ ]:
def crop_img(number_picture: int, constellation_name: str):
    try:
        os.mkdir(f"preprocessed_1/{constellation_name}")
    except:
        pass
    y1 = 70
    y2 = 75
    x = 0
    w = 2561
    h = 1536
    for x in range(0, number_picture, 1):
        image = cv2.imread(
            f"./constellations/{constellation_name}/{constellation_name}{x}.png"
        )
        crop = image[y1 : -y2 + h, x : x + w]
        cv2.imwrite(
            f"./preprocessed_1/{constellation_name}/{constellation_name}{x}.png", crop
        )

In [ ]:
crop_img(46, "leo")

In [ ]:
crop_img(46, "orion")

### Informations sur les coordonnées choisis pour rogner les images   
   
y1 représente la valeur comprise entre la valeur maximale de l'axe x,y et la nouvelle valeur maximale de l'axe x,y.    
y1 permet de rogner la hauteur haute de l'image.
   
y2 représente la valeur qui se situe entre l'origine de l'axe x,y et la valeur de la nouvelle origine de l'axe x,y.   
y2 permet de rogner la hauteur basse de l'image.   
   

Pour connaitre les nouvelles valeurs souhaitées (y1 et y2), il a fallu ouvrir une image du dataset sous le logiciel Paint.  
Une fois ouverte, il a fallu activer le quadrillage et utiliser l'échelle pour connaître le nombre de pixels compris entre les valeurs d'origines et les valeurs souhaitées.  
Les images étant toutes de la même dimension, il a ensuite fallu généraliser un unique rognage pour toutes les images.

## Suppression du texte + rezise

In [ ]:
try:
    os.mkdir("preprocessed_1")
except:
    pass

In [ ]:
def midpoint(x1, y1, x2, y2):
    x_mid = int((x1 + x2) / 2)
    y_mid = int((y1 + y2) / 2)
    return (x_mid, y_mid)


pipeline = keras_ocr.pipeline.Pipeline()


def inpaint_text(pipeline, constellation_name: str, number_picture: int):
    try:
        os.mkdir(f"preprocessed_1/{constellation_name}")
    except:
        pass
    # read image
    for x in range(0, number_picture, 1):
        img = keras_ocr.tools.read(
            f"./preprocessed_1/{constellation_name}/{constellation_name}{x}.png"
        )
        # generate (word, box) tuples
        prediction_groups = pipeline.recognize([img])
        mask = np.zeros(img.shape[:2], dtype="uint8")
        for box in prediction_groups[0]:
            x0, y0 = box[1][0]
            x1, y1 = box[1][1]
            x2, y2 = box[1][2]
            x3, y3 = box[1][3]

            x_mid0, y_mid0 = midpoint(x1, y1, x2, y2)
            x_mid1, y_mi1 = midpoint(x0, y0, x3, y3)

            thickness = int(math.sqrt((x2 - x1) ** 2 + (y2 - y1) ** 2))

            cv2.line(mask, (x_mid0, y_mid0), (x_mid1, y_mi1), 255, thickness)
            img = cv2.inpaint(img, mask, 7, cv2.INPAINT_NS)
            # img_resize = cv2.resize(img, dsize=(780,423))
            img_resize = cv2.resize(img, dsize=(256, 256))
        cv2.imwrite(
            f"./preprocessed_1/{constellation_name}/{constellation_name}{x}.png",
            cv2.cvtColor(img_resize, cv2.COLOR_RGB2BGR),
        )

In [ ]:
inpaint_text(pipeline, "leo", 46)

In [ ]:
inpaint_text(pipeline, "orion", 46)

J'ai essayé d'utiliser tesseract mais la librairie est compliquée à installer (setting de path non mentionné, fichiers manquants) et par manque de temps, je vais utiliser keras-ocr qui ne pose pas de problème lors de l'installation car appartenant au framework keras et qui possède un tuto permettant de supprimer des mots sur une image.

## Convertion en noir et blanc

In [37]:
try:
    os.mkdir("preprocessed_2")
except:
    pass

In [38]:
def convert2gray(constellation_name: str, number_picture: int):
    try:
        os.mkdir(f"preprocessed_2/{constellation_name}")
    except:
        pass
    # read image
    for x in range(0, number_picture, 1):
        originalImage = cv2.imread(
            f"./preprocessed_1/{constellation_name}/{constellation_name}{x}.png"
        )
        grayImage = cv2.cvtColor(originalImage, cv2.COLOR_BGR2GRAY)
        cv2.imwrite(
            f"./preprocessed_2/{constellation_name}/{constellation_name}{x}.png",
            grayImage,
        )

In [39]:
convert2gray("leo", 46)

In [40]:
convert2gray("orion", 46)

## Star extraction

In [160]:
try:
    os.mkdir("preprocessed_3")
except:
    pass

In [161]:
def extract_star(constellation_name: str, number_picture: int):
    try:
        os.mkdir(f"preprocessed_3/{constellation_name}")
    except:
        pass

    for x in range(0, number_picture, 1):
        # reading image
        img = cv2.imread(
            f"./preprocessed_1/{constellation_name}/{constellation_name}{x}.png"
        )

        # converting image into grayscale image
        gray = cv2.cvtColor(img, cv2.COLOR_BGR2GRAY)

        # setting threshold of gray image
        _, threshold = cv2.threshold(gray, 127, 255, cv2.THRESH_BINARY)

        # find contours
        contours, _ = cv2.findContours(
            threshold, cv2.RETR_TREE, cv2.CHAIN_APPROX_SIMPLE
        )

        # create mask
        mask = np.zeros(img.shape[:2], np.uint8)

        # draw contours
        cv2.drawContours(mask, contours, -1, 255, -1)

        # crop mask
        dst = cv2.bitwise_and(img, img, mask=mask)
        cv2.imwrite(
            f"./preprocessed_3/{constellation_name}/{constellation_name}{x}.png", dst
        )

In [162]:
extract_star("leo", 46)

In [163]:
extract_star("orion", 46)